In [111]:
import java.nio.file.Files
import java.nio.file.Path

data class BenchmarkData(val name: String, val score: Double, val error: Double) {
    override fun toString(): String = "$name: $score ± $error ns/op"
}

val benchmarkLines = Files.readAllLines(Path.of("build", "results", "jmh", "results.txt"))
    .mapNotNull {
        val r = Regex("JmhBenchmark\\.([^ ]+) +avgt +\\d+ +([0-9.]+) +± +([0-9.]+) +ns/op")
        val match = r.matchEntire(it) ?: return@mapNotNull null
        BenchmarkData(
            match.groups[1]!!.value,
            match.groups[2]!!.value.toDouble(),
            match.groups[3]!!.value.toDouble(),
        )
    }
benchmarkLines.joinToString("\n")

baselineDouble: 412211.953 ± 2577.468 ns/op
baselineFloat: 354736.155 ± 11537.748 ns/op
inlineDouble: 369105.338 ± 3491.826 ns/op
inlineFloat: 356722.872 ± 12582.934 ns/op
longPackFloat: 586244.873 ± 37386.012 ns/op
mutableRefDouble: 365144.456 ± 2842.053 ns/op
mutableRefFloatIn1Long: 565775.282 ± 2495.777 ns/op
mutableRefFloatIn2Longs: 353739.675 ± 1365.508 ns/op
valueDouble: 247183.059 ± 1857.298 ns/op
valueFloat: 248457.325 ± 2510.454 ns/op
valueInlineDouble: 251691.341 ± 704.228 ns/op
valueInlineFloat: 255053.067 ± 5392.743 ns/op

In [112]:
%use lets-plot

In [113]:
import org.jetbrains.letsPlot.intern.PosKind
import org.jetbrains.letsPlot.intern.StatKind
import org.jetbrains.letsPlot.intern.layer.PosOptions
import org.jetbrains.letsPlot.intern.layer.StatOptions

val names = benchmarkLines.map { it.name }
val scores = benchmarkLines.map { it.score }
val scoresMinusErrors = benchmarkLines.map { it.score - it.error / 2 }
val scoresPlusErrors = benchmarkLines.map { it.score + it.error / 2 }
val baseline = listOf("Float", "Double").associateWith { type ->
    benchmarkLines.single { it.name == "baseline$type" }
}
val plot = letsPlot() +
        labs("Microbenchmark of 3D scene rendering", fill = "Benchmark", y = "ns/op") +
        geomBar(stat = StatOptions(kind = StatKind.IDENTITY)) {
            x = names
            y = scores
            fill = names
        } +
        geomErrorBar(
            width = 0.5,
            position = PosOptions(kind = PosKind.DODGE),
            size = 0.6
        ) {
            x = names
            ymin = scoresMinusErrors
            ymax = scoresPlusErrors
        } +
        geomText(
            stat = StatOptions(kind = StatKind.IDENTITY),
            position = positionStack(vjust = 0.5),
            size = 5,
        ) {
            x = names
            y = scores
            label = benchmarkLines.map {
                val isFloat = it.name.contains("Float")
                val type = if (isFloat) "Float" else "Double"
                val score = it.score / baseline[type]!!.score
                val error = it.error / baseline[type]!!.score
                "${String.format("%.0f", score * 100)}%\n±\n${String.format("%.1f", error * 100)}%\n${type}"
            }
        } +
        scaleXDiscrete(
            labels = names,
            name = "",
            limits = names + listOf(""),
            breaks = names.toList()
        ) +
        scaleYContinuous(limits = 0 to scoresPlusErrors.max(), name = "ns/op") +
        theme(plotTitle = elementText(hjust = 0.5)).legendPositionRight() +
        ggsize(1000, 800)


plot.show()

In [114]:
ggsave(plot = plot, filename = "results.svg")

/Users/Evgeniy.Zhelenskiy/IdeaProjects/Wolf3d/lets-plot-images/results.svg